In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First infernece is done on the salesforc/blip-vqa-base model on 30k models, it is just the baseline evaluation of the model without hypertuning.

In [ ]:
import pandas as pd
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import os
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

df = pd.read_csv('/kaggle/input/actual-dataset/merged_data_actual.csv')
dataset_dir = '/kaggle/input/images/abo-images-small/images/small'

df = df.sample(n=30000, random_state=42).reset_index(drop=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

results = []
smoothie = SmoothingFunction().method1
bleu_scores = []

for i in tqdm(range(len(df))):
    row = df.iloc[i]
    img_path = os.path.join(dataset_dir, row['path'])

    try:
        image = Image.open(img_path).convert("RGB")
    except:
        print(f"Could not open image {img_path}")
        continue

    question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
    true_answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""

    question += " (answer in one word)"

    inputs = processor(image, question, return_tensors="pt").to(device)
    output = model.generate(**inputs,temperature=0.7)
    pred = processor.decode(output[0], skip_special_tokens=True).strip()

    reference = [true_answer.lower().split()]
    candidate = pred.lower().split()
    bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu_scores.append(bleu)

    results.append({
        'Index': i,
        'Question': question,
        'GroundTruth': true_answer,
        'Prediction': pred,
        'BLEU-1': bleu
    })

res_df = pd.DataFrame(results)
res_df['Match'] = res_df.apply(
    lambda x: x['GroundTruth'].strip().lower() == x['Prediction'].strip().lower(),
    axis=1
)

accuracy = res_df['Match'].mean()
average_bleu = sum(bleu_scores) / len(bleu_scores)

res_df.to_csv('/kaggle/working/blip_vqa_base_30k_sample_results.csv', index=False)
print(f"BLIP-VQA-Base Exact Match Accuracy: {accuracy:.4f}")
print(f"BLIP-VQA-Base Average BLEU-1 Score: {average_bleu:.4f}")
res_df.head(10)


This code gave a bleu score of 0.25 only, It has used word to word matchings. 

Next we tried to use the other model, 

In [ ]:
import pandas as pd
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import os
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

df = pd.read_csv('/kaggle/input/actual-dataset/merged_data_actual.csv')
dataset_dir = '/kaggle/input/images/abo-images-small/images/small'


df = df.sample(n=30000, random_state=42).reset_index(drop=True)


from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    device_map="auto",
    quantization_config=bnb_config
)


results = []
smoothie = SmoothingFunction().method1
bleu_scores = []

for i in tqdm(range(len(df))):
    row = df.iloc[i]
    img_path = os.path.join(dataset_dir, row['path'])

    try:
        image = Image.open(img_path).convert("RGB")
    except:
        print(f"Could not open image {img_path}")
        continue

    question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
    true_answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""

    question += " (answer in one word)"

    inputs = processor(images=image, text=question, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=20)
    pred = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    reference = [true_answer.lower().split()]
    candidate = pred.lower().split()
    bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu_scores.append(bleu)

    results.append({
        'Index': i,
        'Question': question,
        'GroundTruth': true_answer,
        'Prediction': pred,
        'BLEU-1': bleu
    })


res_df = pd.DataFrame(results)
res_df['Match'] = res_df.apply(
    lambda x: x['GroundTruth'].strip().lower() == x['Prediction'].strip().lower(),
    axis=1
)

accuracy = res_df['Match'].mean()
average_bleu = sum(bleu_scores) / len(bleu_scores)


res_df.to_csv('/kaggle/working/blip2_flan_t5_xl_30k_results.csv', index=False)
print(f"BLIP-2 FLAN-T5-XL Exact Match Accuracy: {accuracy:.4f}")
print(f"BLIP-2 FLAN-T5-XL Average BLEU-1 Score: {average_bleu:.4f}")
res_df.head(10)


We got the same results , but this is a model with higher no of parameters , so we choose to go with the smaller model blip-vqa-base of around 400 m parameters, next we applied the finetuning technique.

In [ ]:
import pandas as pd
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from PIL import Image
import os
from torch.utils.data import Dataset
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

# Download NLTK data (if not already present)
nltk.download('punkt')

# Custom Dataset Class
class VQADataset(Dataset):
    def __init__(self, df, processor, dataset_dir):
        self.df = df
        self.processor = processor
        self.dataset_dir = dataset_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.dataset_dir, row['path'])
        
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            return None
            
        question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
        answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""
        
        # Format the question to encourage short answers
        question += " (answer in one word)"
        
        # Process the image and text
        inputs = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding="max_length",
            max_length=32,
            truncation=True
        )
        
        # Process the answer
        answer_inputs = self.processor(
            text=answer,
            return_tensors="pt",
            padding="max_length",
            max_length=32,
            truncation=True
        )
        
        # Remove batch dimension and convert to numpy
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = answer_inputs["input_ids"].squeeze(0)
        
        return inputs

# Load and prepare data
df = pd.read_csv('/kaggle/input/actual-dataset/merged_data_actual.csv')
dataset_dir = '/kaggle/input/images/abo-images-small/images/small'

# Take a subset of 10k images
df = df.sample(n=10000, random_state=42).reset_index(drop=True)

# Split into train and eval (90%/10%)
train_df = df.sample(frac=0.9, random_state=42)
eval_df = df.drop(train_df.index)

# Initialize processor and model
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

# Define LoRA configuration
# Targeting key layers in BLIP architecture:
# - text encoder's query/value layers
# - vision encoder's attention layers
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    target_modules=["query", "value", "vision_encoder.encoder.layer.*.attention.attention"],  # Key layers to adapt
    lora_dropout=0.05,
    bias="none",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Create datasets
train_dataset = VQADataset(train_df, processor, dataset_dir)
eval_dataset = VQADataset(eval_df, processor, dataset_dir)

# Filter out None values (failed image loads)
train_dataset = [x for x in train_dataset if x is not None]
eval_dataset = [x for x in eval_dataset if x is not None]

# Training arguments
training_args = TrainingArguments(
    output_dir="./blip-vqa-lora",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    fp16=True,
    save_steps=1000,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500,
    remove_unused_columns=False,
    report_to="none",
)

# Custom compute_metrics function for evaluation
def compute_metrics(p):
    predictions, labels = p
    decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)
    
    # Calculate BLEU and accuracy
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    exact_matches = []
    
    for pred, label in zip(decoded_preds, decoded_labels):
        reference = [label.lower().split()]
        candidate = pred.lower().split()
        bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu_scores.append(bleu)
        exact_matches.append(label.strip().lower() == pred.strip().lower())
    
    return {
        "bleu": np.mean(bleu_scores),
        "accuracy": np.mean(exact_matches),
    }

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./blip-vqa-lora-final")
processor.save_pretrained("./blip-vqa-lora-final")

# Evaluate after training
final_results = trainer.evaluate()
print("Final evaluation results:", final_results)

# Generate predictions on test set for comparison
test_results = []
bleu_scores = []
smoothie = SmoothingFunction().method1 
model.eval()
for i in tqdm(range(len(eval_df))):
    row = eval_df.iloc[i]
    img_path = os.path.join(dataset_dir, row['path'])
    
    try:
        image = Image.open(img_path).convert("RGB")
    except:
        continue
        
    question = str(row['Question']).strip() if pd.notnull(row['Question']) else ""
    true_answer = str(row['Answer']).strip() if pd.notnull(row['Answer']) else ""
    
    question += " (answer in one word)"
    
    inputs = processor(image, question, return_tensors="pt").to(device)
    output = model.generate(**inputs)
    pred = processor.decode(output[0], skip_special_tokens=True).strip()
    
    reference = [true_answer.lower().split()]
    candidate = pred.lower().split()
    bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu_scores.append(bleu)
    
    test_results.append({
        'Question': question,
        'GroundTruth': true_answer,
        'Prediction': pred,
        'BLEU-1': bleu,
        'Match': true_answer.strip().lower() == pred.strip().lower()
    })

# Convert to DataFrame and evaluate
test_res_df = pd.DataFrame(test_results)
accuracy = test_res_df['Match'].mean()
average_bleu = sum(bleu_scores) / len(bleu_scores)

# Save and show results
test_res_df.to_csv('/kaggle/working/blip_vqa_lora_finetuned_results.csv', index=False)
print(f"LoRA Fine-tuned Model Exact Match Accuracy: {accuracy:.4f}")
print(f"LoRA Fine-tuned Model Average BLEU-1 Score: {average_bleu:.4f}")
test_res_df.head(10)

THis finetuning using just lora has increased the bleu score by 0.10 for 10k samples, so we tried lora with quantisation to speed up the training process. 

In [ ]:
import os
import pandas as pd
import numpy as np
import nltk


nltk.download('punkt')

df = pd.read_csv('/kaggle/input/actual-dataset/final.csv')
df = df.sample(n=10000, random_state=42).reset_index(drop=True)


dataset_dir = '/kaggle/input/images/abo-images-small/images/small'


train_frac, val_frac = 0.8, 0.1
train_end = int(train_frac * len(df))
val_end = train_end + int(val_frac * len(df))

train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Now the datasets (train_df, val_df, test_df) are ready for reuse in other cells

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from PIL import Image
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from transformers import BitsAndBytesConfig
import evaluate
import warnings
import logging
import pandas as pd
import numpy as np
import torch.nn.functional as F


logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message=".Empty candidate sentence detected.")
warnings.filterwarnings("ignore", message=".Empty reference sentence detected.")
warnings.filterwarnings("ignore", message="Some weights of.were not initialized.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class VQADataset(Dataset):
    def _init_(self, df, processor, dataset_dir):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.dataset_dir = dataset_dir

    def _len_(self):
        return len(self.df)

    def _getitem_(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.dataset_dir, row['path'])

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            return self.getitem((idx + 1) % len(self.df))  # Retry another

        question = str(row['Question']).strip() + " (answer in one word)"
        answer = str(row['Answer']).strip()

        inputs = self.processor(images=image, text=question, return_tensors="pt",
                                padding="max_length", max_length=32, truncation=True)
        answer_inputs = self.processor(text=answer, return_tensors="pt",
                                       padding="max_length", max_length=32, truncation=True)

        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = answer_inputs["input_ids"].squeeze(0)
        return inputs


processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base", 
    quantization_config=quant_config,
    device_map="auto"
)


lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "vision_encoder.encoder.layer.*.attention.attention"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


train_dataset = VQADataset(train_df, processor, dataset_dir)
val_dataset = VQADataset(val_df, processor, dataset_dir)
test_dataset = VQADataset(test_df, processor, dataset_dir)


training_args = TrainingArguments(
    output_dir="./blip-vqa-lora",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    fp16=True,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500,
    remove_unused_columns=False,
    report_to="none"
)


def compute_metrics(p):
    predictions, labels = p
    decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

    smoothie = SmoothingFunction().method1
    bleu_scores = []
    exact_matches = []

    for pred, label in zip(decoded_preds, decoded_labels):
        reference = [label.lower().split()]
        candidate = pred.lower().split()
        bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu_scores.append(bleu)
        exact_matches.append(label.strip().lower() == pred.strip().lower())

    return {
        "bleu": np.mean(bleu_scores),
        "accuracy": np.mean(exact_matches),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()


model.save_pretrained("./blip-vqa-lora-final")
processor.save_pretrained("./blip-vqa-lora-final")


bertscore = evaluate.load("bertscore")
test_results = []
bleu_scores = []
bertscore_precisions, bertscore_recalls, bertscore_f1s = [], [], []
bertscore_f1_cosines = []
smoothie = SmoothingFunction().method1
model.eval()

for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]
    img_path = os.path.join(dataset_dir, row['path'])

    try:
        image = Image.open(img_path).convert("RGB")
    except:
        continue

    question = str(row['Question']).strip() + " (answer in one word)"
    true_answer = str(row['Answer']).strip()

    inputs = processor(image, question, return_tensors="pt").to(device)
    output = model.generate(**inputs)
    pred = processor.decode(output[0], skip_special_tokens=True).strip()

    reference = [true_answer.lower().split()]
    candidate = pred.lower().split()
    bleu = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu_scores.append(bleu)

    try:
        bert_out = bertscore.compute(predictions=[pred], references=[true_answer], lang="en", model_type="microsoft/deberta-xlarge-mnli", return_hash=False, verbose=False)
        bert_p = bert_out['precision'][0]
        bert_r = bert_out['recall'][0]
        bert_f1 = bert_out['f1'][0]

       
        ref_vec = torch.tensor(bert_out['ref_embed'][0])
        pred_vec = torch.tensor(bert_out['pred_embed'][0])
        cosine_sim = F.cosine_similarity(ref_vec, pred_vec, dim=0).item()

    except:
        bert_p = bert_r = bert_f1 = cosine_sim = 0.0

    bertscore_precisions.append(bert_p)
    bertscore_recalls.append(bert_r)
    bertscore_f1s.append(bert_f1)
    bertscore_f1_cosines.append(cosine_sim)

    test_results.append({
        'Question': question,
        'GroundTruth': true_answer,
        'Prediction': pred,
        'BLEU-1': bleu,
        'BERTScore_Precision': bert_p,
        'BERTScore_Recall': bert_r,
        'BERTScore_F1': bert_f1,
        'BERTScore_F1_Cosine': cosine_sim,
        'Match': true_answer.strip().lower() == pred.strip().lower()
    })


test_res_df = pd.DataFrame(test_results)
accuracy = test_res_df['Match'].mean()
average_bleu = np.mean(bleu_scores)
average_bert_f1 = np.mean(bertscore_f1s)
average_bert_f1_cosine = np.mean(bertscore_f1_cosines)

test_res_df.to_csv('/kaggle/working/blip_vqa_lora_finetuned_results_with_bertscore.csv', index=False)

print(f"\n Final Test Accuracy: {accuracy:.4f}")
print(f"Final BLEU-1 Score: {average_bleu:.4f}")
print(f"Final BERTScore F1: {average_bert_f1:.4f}")
print(f"BERTScore F1 Cosine Similarity: {average_bert_f1_cosine:.4f}")
print(test_res_df.head(10))

THis is the final finetuning code, it is actually, it uses the lora with quantisation, with this we achieved a bleu score of around 0.44 and bert score around 0.93.